Requires openpyxl

In [4]:
!pwd
os.chdir(EXPERIMENT_HOME)

/opt/workspace


In [5]:
import sys, os, os.path
from sqlalchemy import create_engine
engine = create_engine('sqlite:///dataset/stock.db', echo=False)
engine.table_names()

['DividendYield',
 'MarketValue',
 'TurnoverByValue',
 'TurnoverByVolume',
 'auth_group',
 'auth_group_permissions',
 'auth_permission',
 'auth_user',
 'auth_user_groups',
 'auth_user_user_permissions',
 'dataset_set100',
 'dataset_set50',
 'dataset_stock',
 'dataset_stock_set100_membership_periods',
 'dataset_stock_set50_membership_periods',
 'dataset_timeseries',
 'django_admin_log',
 'django_content_type',
 'django_migrations',
 'django_session',
 'litreview_literature',
 'litreview_literature_refgroups',
 'litreview_refgroup',
 'set100_companies',
 'set100_company_dim',
 'set100_daily_fact',
 'set100_daily_facts',
 'set100_deadlist',
 'set100_time_dim',
 'set100_updates',
 'sqlite_sequence']

In [ ]:
import sqlite3
import pandas as pd

DB_FILE = 'dataset/stock.db'
cnx = sqlite3.connect(DB_FILE)
# pd.read_sql_query("SELECT * FROM dataset_set50", cnx)

## List of SET100 updates

The index had 15 updates during the study period of 2015-2019. These are the records from update_id=Update_3 to update_id=Update_17.

In [ ]:
# cnx.commit()
# cnx.close()
df_uplist = pd.read_sql("SELECT update_id AS update_seq, date_from, date_to, date_published FROM dataset_set100", cnx)
df_uplist

In [ ]:
import json
addlist = json.loads(open("dataset/set100_add_list.json", "r").read())
df_addlist = pd.DataFrame()
for k, v in addlist.items():
    inc = ",".join(v)
    print(k,inc)
    df_addlist = df_addlist.append({"update_seq": k, "stocks_added": inc}, ignore_index=True)
# df_addlist

Stocks in Update_3


In [ ]:
df_up3 = pd.read_sql("SELECT constituents FROM dataset_set100 WHERE update_id='Update_3'", cnx)
up3_stocks = df_up3[0:1].values[0][0].replace(" ",",")
print("Update_3 " + up3_stocks)

In [ ]:
up = {}
for i in range(3,18):
    df_up = pd.read_sql("SELECT constituents FROM dataset_set100 WHERE update_id='Update_%s'" % i, cnx)
    stocks = set(df_up[0:1].values[0][0].replace(" ",",").split(','))
    up[i] = stocks
up

In [ ]:
up[4].difference(up[3]) # Added in 3

In [ ]:
up[3].difference(up[4]) # Removed in 3

In [ ]:
up[16].difference(up[17])

In [ ]:
len(up[17])

In [ ]:
df_addlist.loc[df_addlist["update_seq"]=='Update_3','stocks_added'] = up3_stocks
df_addlist[["update_seq", "stocks_added"]]

In [ ]:
# Union all stocks

In [ ]:
union_set = set()
for r in df_addlist["stocks_added"]:
    # print(r.split(","))
    union_set.update(r.split(","))
len(union_set)

In [ ]:
df_stocks = pd.DataFrame(sorted(list(union_set)), columns=['ticker'])
df_stocks.index = range(1, len(df_stocks) + 1)
df_stocks

# Save Output



In [ ]:
!pip install 

In [ ]:
# See documentation https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_excel.html

with pd.ExcelWriter("SET100.xlsx") as writer:
    df_uplist.to_excel(writer, sheet_name="1-UpdateSeq")
    df_addlist[["update_seq", "stocks_added"]].to_excel(writer, sheet_name="2-StocksAdded")
    df_stocks.to_excel(writer, sheet_name="3-AllStocks")

# In-Memory Dataframe

Get stocks in the dataset folder. The stocks are indexed in _directory.csv.

In [ ]:
os.chdir("/home/jovyan/dataset/set100.data")
stocks = pd.read_csv('_directory.csv', index_col=0)
print("Stocks count:", len(stocks))
stocks.tail()

Construct an in-memory dataframe containing all stocks time-series.

In [ ]:
period_start='2015-01-01'
period_end='2019-12-31'
series = {}

# Set index for data series to the date column
# Add column ticker, sector, industry, observation count 
#   to each data series and combine to single dictionary
for s in stocks.symbol.values:
    df = pd.read_csv("%s.csv" % s, index_col=0)
    df.loc[:,('Date')] = df.index
    df['Ticker'] = s
    sector = stocks[stocks['symbol']==s].sector.values[0]
    df['Sector'] = sector
    industry = stocks[stocks['symbol']==s].industry.values[0]
    df['Industry'] = industry
    df['Observations'] = len(df)
    series[s] = df[period_start:period_end]
# The dictionary 'series' contains time-series with key ticker.
# Stack dataframes together as one dataframe.
rows = []
for k, d in series.items():
    rows.append(d)
df = pd.concat(rows, ignore_index=True)

In [ ]:
# Pivot table: Value=Observations; the number of data observations for the stock.
ptable = df.pivot_table(values='Observations', index='Date', columns=['Industry', 'Sector', 'Ticker'])
ptable.tail()

In [ ]:
# Observations count at last date of the dataset.
table_7 = ptable['2019-12-31':].T
table_7

In [ ]:
# table_7.to_csv('table_7.csv')

In [ ]:

# table = df.pivot(columns='Ticker')['Observations'] # Pivot table
# table

# table_7.pivot()['Ticker']

In [ ]:

# table = df.pivot(columns='Ticker')['Observations'] # Pivot table
# table

# table_7.pivot()['Ticker']

In [ ]:
# output = os.path.join(os.environ['EXPERIMENT_HOME'],'tables','table_7')
# table_7.to_csv(output)

# table_7[['Ticker']].index
table_7.columns

In [ ]:
# Get ticker from multiindex.
# tickers = np.array([i[2] for i in table_7.index.values])

fig, ax = plt.subplots(figsize=(8,18))
table_7.plot.barh(ax=ax, legend=False) # , xticks=tickers to change ticker

ax.tick_params(axis='y', which='major', labelsize=10)
ax.tick_params(axis='y', which='minor', labelsize=8)
plt.xlabel('Observations Count (Closing Price)')
plt.title("SET100 Stocks: Closing price observations during 2015-2019")